In [1]:
%tensorflow_version 1.x

!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_deepct.git


TensorFlow 1.x selected.
  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-ecb2i1mk/python-terrier
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-ecb2i1mk/python-terrier
     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 1.1MB 9.8MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 163kB 20.9MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 1.8MB 22.5MB/s 
     |████████████████████████████████| 5.5MB 45.9MB/s 
     |████████████████████████████████| 133kB 49.9MB/s 
     |████████████████████████████████| 645kB 56.3MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.4.0-cp37-none-any.whl size=78539 sha256=a74eada0689f8b28e4354c3a5bbac7f959cdd143826ba633e17ff638386fce98
  Stored in directory: /tmp/pip-ephem-wh

In [2]:
import pyterrier as pt
pt.init()

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
assert tf.__version__.startswith("1"), "TF 1 is required by DeepCT; on Colab, use %tensorflow_version 1.x"
tf.logging.set_verbosity(tf.logging.ERROR)

terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.4.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
import pyterrier_deepct
import os

In [4]:
if not os.path.exists("marco.zip"):
  !wget http://boston.lti.cs.cmu.edu/appendices/arXiv2019-DeepCT-Zhuyun-Dai/outputs/marco.zip
  !unzip marco.zip

--2021-03-19 22:57:07--  http://boston.lti.cs.cmu.edu/appendices/arXiv2019-DeepCT-Zhuyun-Dai/outputs/marco.zip
Resolving boston.lti.cs.cmu.edu (boston.lti.cs.cmu.edu)... 128.2.207.139, 128.2.207.32
Connecting to boston.lti.cs.cmu.edu (boston.lti.cs.cmu.edu)|128.2.207.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1237248594 (1.2G) [application/zip]
Saving to: ‘marco.zip’

marco.zip           100%[===================>]   1.15G  39.2MB/s    in 31s     

2021-03-19 22:57:38 (38.6 MB/s) - ‘marco.zip’ saved [1237248594/1237248594]

Archive:  marco.zip
   creating: marco/
  inflating: marco/checkpoint        
  inflating: marco/graph.pbtxt       
  inflating: marco/model.ckpt-65816.data-00000-of-00001  
  inflating: marco/model.ckpt-65816.index  
  inflating: marco/model.ckpt-65816.meta  
  inflating: marco/train.tf_record   


In [5]:
if not os.path.exists("uncased_L-12_H-768_A-12.zip"):
  !wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
  !unzip uncased_L-12_H-768_A-12.zip
  !mkdir -p bert-base-uncased
  !mv vocab.txt bert_* bert-base-uncased/

--2021-03-19 22:57:53--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 142.250.99.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 408102251 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 389.20M  38.7MB/s    in 10s     

2021-03-19 22:58:03 (38.0 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [408102251/408102251]

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: bert_model.ckpt.data-00000-of-00001  
  inflating: bert_config.json        
  inflating: vocab.txt               
  inflating: bert_model.ckpt.index   


In [6]:
deepct = pyterrier_deepct.DeepCTTransformer("bert-base-uncased/bert_config.json", "marco/model.ckpt-65816")

In [7]:
import pandas as pd
df = pd.DataFrame([{"docno" : "d1", "text" :"this is a test document"}])
df

,docno,text
0,d1,this is a test document


In [8]:
deepct.transform(df).iloc[0]["text"]

,docno,text
0,d1,this this this this this is test test test tes...


In [9]:
dataset = pt.get_dataset("irds:vaswani")
index_loc = "./deepct_index_path"
indexer = deepct >> pt.IterDictIndexer(index_loc)
indexref = indexer.index(dataset.get_corpus_iter())


[INFO] [starting] building docstore
[INFO] If you have a local copy of http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/23e5607081191b153738e81fbd834680
[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
docs_iter: 0it [00:00, ?it/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.0%| 0.00/2.13M [00:00<?, ?B/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 0.8%| 16.4k/2.13M [00:00<00:16, 129kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 2.3%| 49.2k/2.13M [00:00<00:11, 186kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 4.2%| 90.1k/2.13M [00:00<00:09, 225kB/s]
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 6.9%| 147k/2.13M [00:00<00:07, 274kB/s] 
http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: 14.6%| 311k/2.13M [00:00<00:03, 

vaswani documents:   0%|          | 0/11429 [00:00<?, ?it/s]

In [12]:
pt.Experiment([
  pt.BatchRetrieve(pt.get_dataset("vaswani").get_index(), wmodel="BM25"),
  pt.BatchRetrieve(indexref, wmodel="BM25"),
  ],
  dataset.get_topics(),
  dataset.get_qrels(),
  names=["BM25", "BM25_deepct"],
  eval_metrics=["map", "recip_rank"]
)

,name,map,recip_rank
0,BM25,0.296517,0.725665
1,BM25_deepct,0.296275,0.706092
